In [3]:
#import dependencies
import pandas as pd
import numpy as np
import requests
import json
import random
import matplotlib.pyplot as plt


In [4]:
#path to CSV file
csv_path="2015_census_tract_data.csv"

#read CSV file
Census_file=pd.read_csv(csv_path)


In [5]:
#check to see if there are rows with missing data
Census_file.count()

CensusTract        74001
State              74001
County             74001
TotalPop           74001
Men                74001
Women              74001
Hispanic           73311
White              73311
Black              73311
Native             73311
Asian              73311
Pacific            73311
Citizen            74001
Income             72901
IncomeErr          72901
IncomePerCap       73261
IncomePerCapErr    73261
Poverty            73166
ChildPoverty       72883
Professional       73194
Service            73194
Office             73194
Construction       73194
Production         73194
Drive              73204
Carpool            73204
Transit            73204
Walk               73204
OtherTransp        73204
WorkAtHome         73204
MeanCommute        73052
Employed           74001
PrivateWork        73194
PublicWork         73194
SelfEmployed       73194
FamilyWork         73194
Unemployment       73199
dtype: int64

In [6]:
#remove rows with missing data
clean_census_file=Census_file.dropna(how="any")
clean_census_file.count()

CensusTract        72727
State              72727
County             72727
TotalPop           72727
Men                72727
Women              72727
Hispanic           72727
White              72727
Black              72727
Native             72727
Asian              72727
Pacific            72727
Citizen            72727
Income             72727
IncomeErr          72727
IncomePerCap       72727
IncomePerCapErr    72727
Poverty            72727
ChildPoverty       72727
Professional       72727
Service            72727
Office             72727
Construction       72727
Production         72727
Drive              72727
Carpool            72727
Transit            72727
Walk               72727
OtherTransp        72727
WorkAtHome         72727
MeanCommute        72727
Employed           72727
PrivateWork        72727
PublicWork         72727
SelfEmployed       72727
FamilyWork         72727
Unemployment       72727
dtype: int64

In [7]:
#column names
clean_census_file.columns

Index(['CensusTract', 'State', 'County', 'TotalPop', 'Men', 'Women',
       'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [8]:
#create a dataframe from the file
DF_census_file=pd.DataFrame(clean_census_file)
DF_census_file.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


In [9]:
grouped_income = DF_census_file.groupby(["State","County"]).apply(lambda x: np.average(x["Income"],weights=x["TotalPop"]))
grouped_income=pd.DataFrame(grouped_income,columns=["Income"])
grouped_income=grouped_income.reset_index()
grouped_income["ID"] = grouped_income["State"]+grouped_income["County"]
grouped_income.head()

,State,County,Income,ID
0,Alabama,Autauga,51696.286576,AlabamaAutauga
1,Alabama,Baldwin,51074.359577,AlabamaBaldwin
2,Alabama,Barbour,32959.300349,AlabamaBarbour
3,Alabama,Bibb,38886.634888,AlabamaBibb
4,Alabama,Blount,46237.969018,AlabamaBlount


In [10]:
#find total number of people per ethnicity
DF_census_file["Hispanic Population"]=(DF_census_file["Hispanic"]/100)*DF_census_file["TotalPop"]
DF_census_file["White Population"]=(DF_census_file["White"]/100)*DF_census_file["TotalPop"]
DF_census_file["Black Population"]=(DF_census_file["Black"]/100)*DF_census_file["TotalPop"]
DF_census_file["Native Population"]=(DF_census_file["Native"]/100)*DF_census_file["TotalPop"]
DF_census_file["Asian Population"]=(DF_census_file["Asian"]/100)*DF_census_file["TotalPop"]
DF_census_file["Pacific Population"]=(DF_census_file["Pacific"]/100)*DF_census_file["TotalPop"]
DF_census_file["Other Population"]=DF_census_file["TotalPop"]-DF_census_file["Pacific Population"]-DF_census_file["Asian Population"]-DF_census_file["Native Population"]-DF_census_file["Hispanic Population"]-DF_census_file["White Population"]-DF_census_file["Black Population"]
DF_census_file["ID"]=DF_census_file["State"]+DF_census_file["County"]


In [11]:
TractCount=DF_census_file.groupby(["State","County"]).count()["CensusTract"]
TractCount=pd.DataFrame(TractCount)
TractCount=TractCount.reset_index()
TractCount.head()

,State,County,CensusTract
0,Alabama,Autauga,12
1,Alabama,Baldwin,31
2,Alabama,Barbour,9
3,Alabama,Bibb,4
4,Alabama,Blount,9


In [12]:
DF_census_file.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,FamilyWork,Unemployment,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Pacific Population,Other Population,ID
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.0,5.4,17.532,1702.552,149.996,5.844,11.688,0.000,60.388,AlabamaAutauga
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,13.3,17.248,871.024,1149.148,0.000,49.588,0.000,68.992,AlabamaAutauga
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,6.2,0.000,2211.160,552.048,14.840,41.552,8.904,139.496,AlabamaAutauga
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,10.8,464.415,3662.244,163.651,70.768,0.000,0.000,61.922,AlabamaAutauga
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,4.2,75.341,7372.655,2669.224,0.000,408.994,0.000,236.786,AlabamaAutauga


In [13]:
Test_census_file=DF_census_file[["State","County","ID","TotalPop","Men","Women","Hispanic Population","White Population","Black Population","Native Population","Asian Population","Other Population"]]
Test_census_file=Test_census_file.groupby(["State","County","ID"],as_index=False).sum()
Test_census_file.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314


In [14]:
merge_table=pd.merge(Test_census_file,grouped_income,on=["State","County","ID"] ,how="outer")
merge_table.head()


,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population,Income
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898,51696.286576
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128,51074.359577
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398,32959.300349
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569,38886.634888
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314,46237.969018


In [15]:
merge_table=pd.merge(merge_table,TractCount,on=["State","County"],how="outer")
merge_table.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,Asian Population,Other Population,Income,CensusTract
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,528.826,924.898,51696.286576,12
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,1296.962,3284.128,51074.359577,31
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,117.623,491.398,32959.300349,9
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,22.741,301.569,38886.634888,4
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,85.670,895.314,46237.969018,9


In [16]:
#Filter
merge_table["Hispanic Population %"]=merge_table["Hispanic Population"]/merge_table["TotalPop"]*100
merge_table["White Population %"]=merge_table["White Population"]/merge_table["TotalPop"]*100
merge_table["Black  Population %"]=merge_table["Black Population"]/merge_table["TotalPop"]*100
merge_table["Asian Population %"]=merge_table["Asian Population"]/merge_table["TotalPop"]*100
merge_table["Other Population %"]=merge_table["Other Population"]/merge_table["TotalPop"]*100
merge_table["Native Population %"]=merge_table["Native Population"]/merge_table["TotalPop"]*100
merge_table["% Men"]=merge_table["Men"]/merge_table["TotalPop"]*100
merge_table["% Women"]=merge_table["Women"]/merge_table["TotalPop"]*100


In [17]:
merge_table.head()

,State,County,ID,TotalPop,Men,Women,Hispanic Population,White Population,Black Population,Native Population,...,Income,CensusTract,Hispanic Population %,White Population %,Black Population %,Asian Population %,Other Population %,Native Population %,% Men,% Women
0,Alabama,Autauga,AlabamaAutauga,55221,26745,28476,1443.611,41851.017,10230.777,232.967,...,51696.286576,12,2.614243,75.788227,18.526968,0.957654,1.674903,0.421881,48.432661,51.567339
1,Alabama,Baldwin,AlabamaBaldwin,195121,95314,99807,8767.268,162150.656,18530.346,1091.640,...,51074.359577,31,4.493247,83.102616,9.496849,0.664696,1.683124,0.559468,48.848663,51.151337
2,Alabama,Barbour,AlabamaBarbour,26932,14497,12435,1246.497,12451.093,12574.719,50.670,...,32959.300349,9,4.628312,46.231594,46.690625,0.436741,1.824588,0.188141,53.828160,46.171840
3,Alabama,Bibb,AlabamaBibb,22604,12073,10531,499.358,16839.955,4842.938,97.439,...,38886.634888,4,2.209158,74.499889,21.425137,0.100606,1.334140,0.431070,53.410901,46.589099
4,Alabama,Blount,AlabamaBlount,57710,28512,29198,4983.152,50700.459,877.368,168.037,...,46237.969018,9,8.634815,87.853854,1.520305,0.148449,1.551402,0.291175,49.405649,50.594351


In [18]:
merged_clean=merge_table[["State","County","TotalPop","Income","CensusTract","White Population %","Hispanic Population %",
                         "Black  Population %","Asian Population %", "Native Population %","Other Population %", "% Men", "% Women"]]

In [19]:
merged_clean.head()


,State,County,TotalPop,Income,CensusTract,White Population %,Hispanic Population %,Black Population %,Asian Population %,Native Population %,Other Population %,% Men,% Women
0,Alabama,Autauga,55221,51696.286576,12,75.788227,2.614243,18.526968,0.957654,0.421881,1.674903,48.432661,51.567339
1,Alabama,Baldwin,195121,51074.359577,31,83.102616,4.493247,9.496849,0.664696,0.559468,1.683124,48.848663,51.151337
2,Alabama,Barbour,26932,32959.300349,9,46.231594,4.628312,46.690625,0.436741,0.188141,1.824588,53.828160,46.171840
3,Alabama,Bibb,22604,38886.634888,4,74.499889,2.209158,21.425137,0.100606,0.431070,1.334140,53.410901,46.589099
4,Alabama,Blount,57710,46237.969018,9,87.853854,8.634815,1.520305,0.148449,0.291175,1.551402,49.405649,50.594351


In [20]:
#create datasets
#exclude puerto rico
America_County=merged_clean[(merged_clean["State"]!="Puerto Rico")]
#Poor County are counties with incomes less than $25,000
Poor_County=America_County.query("Income<25000")
#Rich County are counties with income more than $75,000
Rich_County=America_County.query("Income>75000")
#Diverse County are counties with no ethnic group making up more than 45% of the population
Diverse_County=America_County[(America_County["White Population %"]<45)& (America_County["Native Population %"]<45)&(America_County["Hispanic Population %"]<45)& (America_County["Black  Population %"]<45)] 
#NonDiverse County are counties where one ethnic group makes up more than 85% of the population with a population greater than 400,000
NonDiverse_County=America_County[(America_County["TotalPop"]>400000)&((America_County["White Population %"]>85)| (America_County["Native Population %"]>85)|(America_County["Hispanic Population %"]>85)| (America_County["Black  Population %"]>85))] 
                              

In [21]:
Poor_County.head()

,State,County,TotalPop,Income,CensusTract,White Population %,Hispanic Population %,Black Population %,Asian Population %,Native Population %,Other Population %,% Men,% Women
17,Alabama,Conecuh,12865,24856.284104,5,50.968916,1.649988,44.681360,0.040365,0.283366,2.376005e+00,48.006218,51.993782
31,Alabama,Greene,8697,20598.498793,3,17.768748,0.715247,80.636185,0.177946,0.079062,6.228125e-01,47.085202,52.914798
52,Alabama,Perry,10038,24035.630305,3,29.987119,0.916956,68.001524,0.450887,0.053626,5.898884e-01,46.333931,53.666069
59,Alabama,Sumter,13341,19387.044449,4,24.822487,0.411933,72.380526,1.697841,0.397354,2.898583e-01,44.262049,55.737951
65,Alabama,Wilcox,11235,24888.263818,4,27.343186,0.593885,72.035514,0.000000,0.027414,3.035697e-15,47.850467,52.149533


In [22]:
#creating CSV Files
Diverse_County.to_csv("Diverse_County.csv")
NonDiverse_County.to_csv("NonDiverse_County.csv")
Poor_County.to_csv("Poor_County.csv")
Rich_County.to_csv("Rich_County.csv")

#APPLYING YELP API

In [90]:

import requests
import pprint
import json
from urllib.parse import quote

api_key="HTxE_2fiZiukf88_6zgw-998n3g0Hp2VCZVttuzh8PYAVGYnMqjyF-20vFikX40klyhOv4-s98YvoQNi0WoNSqRkLhKzc6Go8PCNpheK3map5VsYo69Sl5haqpaoWnYx"

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.

Default_Term="restaurant"
Limit_Count=3



In [24]:
#define function to pull information from yelp
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)
    return response.json()


In [77]:
#function to pull restaurants based by location
def search(api_key,term,location):
#term: the search term (restaurant)
#location: The search location passed to the api
#returns: the json response
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': Limit_Count,
        'sort_by': 'rating'
      }
    return request(API_HOST,SEARCH_PATH,api_key,url_params=url_params)



In [190]:
#create lists to store yelp information
Restaurant_Name=[]
Restaurant_City=[]
Restaurant_State=[]
Restaurant_zip=[]
Restaurant_price_cat=[]
Restaurant_rating=[]
Restaurant_type=[]
Restaurant_County=[]
Restaurant_ReviewCount=[]


In [158]:
Restaurant_Name

[]

In [192]:
#looping through dataset

for index,row in Poor_County.iterrows():
    #print(f"{row['County']}, {row['State']}")
    datatest=search(api_key,"Restaurant",f"{row['County']}, {row['State']}")
    #print(datatest)
    for i in range(Limit_Count):
        #Append Restaurant Name
        try:
            Restaurant_Name.append(datatest["businesses"][i]["name"])
        except len(datatest["businesses"][i]["name"])==0:
            Restaurant_Name.append(0)
        
        #Append Restaurant Zip code
        try:  
            Restaurant_zip.append(datatest["businesses"][i]["location"]["zip_code"])
        except len(Restaurant_zip.append(datatest["businesses"][i]["location"]["zip_code"]))==0:
            Restaurant_zip.append('')  
        
        #appent Restaurant City code
        try:
            Restaurant_City.append(datatest["businesses"][i]["location"]["city"])
        except len(Restaurant_City.append(datatest["businesses"][i]["location"]["city"]))==0:
            Restaurant_City.append('')
        
        #append county
        try:
            Restaurant_County.append(row['County'])
        except:
            Restaurant_County.append(row['County'])
        
        #Append State
        try:
            Restaurant_State.append(datatest["businesses"][i]["location"]["state"])
        except len(Restaurant_State.append(datatest["businesses"][i]["location"]["state"]))==0:
            Restaurant_State.append(row['State'])
        
        #Append Restaurant Rating
        try:
            Restaurant_rating.append(datatest["businesses"][i]["rating"])  
        except len(Restaurant_rating.append(datatest["businesses"][i]["rating"]))==0:
            Restaurant_rating.append('')
        
        #Append Restaurant Type
        try: 
            Restaurant_type.append(datatest["businesses"][i]["categories"][0]["title"])
        except:
            Restaurant_type.append('')
        
        #Append Restaurant Review Count
        try:
            Restaurant_ReviewCount.append(datatest["businesses"][i]["review_count"])
        except:
            Restaurant_ReviewCount.append(0)
        
        #append Restaurant Price
        
        try:
            Restaurant_price_cat.append(datatest["businesses"][i]["price"])
        except KeyError:
            Restaurant_price_cat.append('')
        except len(Restaurant_price_cat.append(datatest["businesses"][i]["price"]))== 0:
            Restaurant_price_cat.append('')
       
                        
#         Restaurant_City.append(datatest["businesses"][i]["location"]["city"])x
#         Restaurant_County.append(row['County'])x
#         Restaurant_State.append(datatest["businesses"][i]["location"]["state"])x
#         Restaurant_zip.append(datatest["businesses"][i]["location"]["zip_code"])x
#         #Restaurant_price_cat.append(datatest["businesses"][i]["price"])
#         Restaurant_rating.append(datatest["businesses"][i]["rating"])x
#         Restaurant_type.append(datatest["businesses"][i]["categories"][0]["title"])x
#         Restaurant_ReviewCount.append(datatest["businesses"][i]["review_count"])x
#         except 
    
  
    
    
    
    
    
    
    
    
#     Yelp_df["Restaurant Name"]=datatest["businesses"][i]["name"]
#         Yelp_df["Restaurant City"]=datatest["businesses"][i]["location"]["city"]
#         Yelp_df[]
        
        
        
        
#         Restaurant_Name.append(datatest["businesses"][i]["name"])
#         Restaurant_City.append(datatest["businesses"][i]["location"]["city"])
#         Restaurant_County.append(row['County'])
#         Restaurant_State.append(datatest["businesses"][i]["location"]["state"])
#         Restaurant_zip.append(datatest["businesses"][i]["location"]["zip_code"])
#         #Restaurant_price_cat.append(datatest["businesses"][i]["price"])
#         Restaurant_rating.append(datatest["businesses"][i]["rating"])
#         Restaurant_type.append(datatest["businesses"][i]["categories"][0]["title"])
#         Restaurant_ReviewCount.append(datatest["businesses"][i]["review_count"])


    
    
#     Yelp_df.set_value(index,"State",data["businesses"][9]["location"]["state"])
#     Yelp_df.set_value(index,"State",data["businesses"][9]["location"]["state"])
    
# print(data["businesses"][9]["location"]["city"])
# print(data["businesses"][9]["location"]["state"])
# print(data["businesses"][9]["location"]["zip_code"])
# print(data["businesses"][9]["price"])
# print(data["businesses"][9]["review_count"])
# print(data["businesses"][9]["rating"])
# print(data["businesses"][9]["categories"][0]["title"])


Querying https://api.yelp.com/v3/businesses/search ...
{'businesses': [{'id': 'the-shack-evergreen', 'name': 'The Shack', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/E8B_1-TkfivWX9IRssXNjw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/the-shack-evergreen?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 30, 'categories': [{'alias': 'southern', 'title': 'Southern'}, {'alias': 'seafood', 'title': 'Seafood'}, {'alias': 'burgers', 'title': 'Burgers'}], 'rating': 5.0, 'coordinates': {'latitude': 31.4595007472441, 'longitude': -86.9695268940034}, 'transactions': [], 'price': '$', 'location': {'address1': '281 Hwy 83', 'address2': '', 'address3': '', 'city': 'Evergreen', 'zip_code': '36401', 'country': 'US', 'state': 'AL', 'display_address': ['281 Hwy 83', 'Evergreen, AL 36401']}, 'phone': '+12515780046', 'display_phone': '(251) 578-0046', 'distance': 7158.932036805807}

{'businesses': [{'id': 'taste-of-india-livingston', 'name': 'Taste of India', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ymipg1BokAqMpqHFBxWT_w/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/taste-of-india-livingston?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 37, 'categories': [{'alias': 'indpak', 'title': 'Indian'}], 'rating': 4.5, 'coordinates': {'latitude': 32.6230090587534, 'longitude': -88.2070376637386}, 'transactions': [], 'price': '$$', 'location': {'address1': '651 Hwy 28 W', 'address2': 'Ste B', 'address3': '', 'city': 'Livingston', 'zip_code': '35470', 'country': 'US', 'state': 'AL', 'display_address': ['651 Hwy 28 W', 'Ste B', 'Livingston, AL 35470']}, 'phone': '+12056522080', 'display_phone': '(205) 652-2080', 'distance': 7624.294460105957}, {'id': 'larkins-restaurant-and-deli-york', 'name': "Larkin's Restaurant & Deli", 'image_url': 'https://

{'businesses': [{'id': 'thai-smiles-waycross', 'name': 'Thai Smiles', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/glChBGwYeXustu7GJf3RSA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/thai-smiles-waycross?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 19, 'categories': [{'alias': 'sushi', 'title': 'Sushi Bars'}, {'alias': 'thai', 'title': 'Thai'}], 'rating': 4.5, 'coordinates': {'latitude': 31.2101, 'longitude': -82.36015}, 'transactions': [], 'price': '$$', 'location': {'address1': '311 Pendleton St', 'address2': '', 'address3': '', 'city': 'Waycross', 'zip_code': '31501', 'country': 'US', 'state': 'GA', 'display_address': ['311 Pendleton St', 'Waycross, GA 31501']}, 'phone': '+19128092348', 'display_phone': '(912) 809-2348', 'distance': 48347.28744307765}, {'id': 'hog-n-bones-waycross', 'name': 'Hog N Bones', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bph

{'businesses': [{'id': 'the-wrigley-taproom-and-brewery-corbin', 'name': 'The Wrigley Taproom & Brewery', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/IDOarWGaur1I6YVxH8U2HQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/the-wrigley-taproom-and-brewery-corbin?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 63, 'categories': [{'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'gastropubs', 'title': 'Gastropubs'}], 'rating': 4.5, 'coordinates': {'latitude': 36.9463914, 'longitude': -84.0961661}, 'transactions': [], 'price': '$$', 'location': {'address1': '207 S Main St', 'address2': '', 'address3': '', 'city': 'Corbin', 'zip_code': '40701', 'country': 'US', 'state': 'KY', 'display_address': ['207 S Main St', 'Corbin, KY 40701']}, 'phone': '+16062612008', 'display_phone': '(606) 261-2008', 'distance': 39804.60119839358}, {'id': 'smokey-bears-bbq-corbin',

{'businesses': [{'id': 'pura-vida-somerset', 'name': 'Pura Vida', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/oRCg3FYwgAgkFg19r_VC2Q/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/pura-vida-somerset?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 53, 'categories': [{'alias': 'latin', 'title': 'Latin American'}], 'rating': 5.0, 'coordinates': {'latitude': 37.044642, 'longitude': -84.626289}, 'transactions': [], 'price': '$$', 'location': {'address1': '3499 S Hwy 27', 'address2': '', 'address3': None, 'city': 'Somerset', 'zip_code': '42501', 'country': 'US', 'state': 'KY', 'display_address': ['3499 S Hwy 27', 'Somerset, KY 42501']}, 'phone': '+16064165559', 'display_phone': '(606) 416-5559', 'distance': 31684.616305423355}, {'id': 'guthries-grill-somerset', 'name': "Guthrie's Grill", 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/FjIr8TQKnIQ8k2DZ_O_TXw/o.jp

{'businesses': [{'id': 'the-onward-store-rolling-fork-4', 'name': 'The Onward Store', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/y_NfWY21Dyhzk8N5xGeYHA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/the-onward-store-rolling-fork-4?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 19, 'categories': [{'alias': 'burgers', 'title': 'Burgers'}], 'rating': 4.5, 'coordinates': {'latitude': 32.9166924, 'longitude': -90.8671314}, 'transactions': [], 'price': '$$', 'location': {'address1': '6693 Hwy 61', 'address2': None, 'address3': '', 'city': 'Rolling Fork', 'zip_code': '39159', 'country': 'US', 'state': 'MS', 'display_address': ['6693 Hwy 61', 'Rolling Fork, MS 39159']}, 'phone': '+16628736809', 'display_phone': '(662) 873-6809', 'distance': 37638.11001395868}, {'id': 'la-fonda-mexican-cantina-and-grill-delhi', 'name': 'La Fonda Mexican Cantina and Grill', 'image_url':

{'businesses': [{'id': 'p-reaux-cajun-mudbugs-and-shrimp-yazoo-city', 'name': 'P-Reaux Cajun Mudbugs & Shrimp', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/-W-3_ZZoEOgDyluGtSoT7w/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/p-reaux-cajun-mudbugs-and-shrimp-yazoo-city?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 14, 'categories': [{'alias': 'cajun', 'title': 'Cajun/Creole'}, {'alias': 'seafood', 'title': 'Seafood'}, {'alias': 'burgers', 'title': 'Burgers'}], 'rating': 4.5, 'coordinates': {'latitude': 32.84746, 'longitude': -90.41651}, 'transactions': [], 'price': '$$', 'location': {'address1': '124 S Water St', 'address2': None, 'address3': None, 'city': 'Yazoo City', 'zip_code': '39194', 'country': 'US', 'state': 'MS', 'display_address': ['124 S Water St', 'Yazoo City, MS 39194']}, 'phone': '+16627464460', 'display_phone': '(662) 746-4460', 'distance': 4352

{'businesses': [{'id': 'kings-tavern-natchez-6', 'name': "King's Tavern", 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/gBWd4YiUxXZHTZp9Lsdojg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/kings-tavern-natchez-6?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 103, 'categories': [{'alias': 'bars', 'title': 'Bars'}, {'alias': 'tradamerican', 'title': 'American (Traditional)'}], 'rating': 4.5, 'coordinates': {'latitude': 31.560381, 'longitude': -91.399106}, 'transactions': ['restaurant_reservation'], 'price': '$$', 'location': {'address1': '613 Jefferson St', 'address2': None, 'address3': '', 'city': 'Natchez', 'zip_code': '39120', 'country': 'US', 'state': 'MS', 'display_address': ['613 Jefferson St', 'Natchez, MS 39120']}, 'phone': '+16014465003', 'display_phone': '(601) 446-5003', 'distance': 38408.00723439243}, {'id': 'old-country-store-lorman-2', 'name': 'Old C

{'businesses': [{'id': 'taqueria-guadalajara-premont', 'name': 'Taqueria Guadalajara', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/reDrEfZV5PF06ip0G90khA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/taqueria-guadalajara-premont?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw', 'review_count': 11, 'categories': [{'alias': 'mexican', 'title': 'Mexican'}], 'rating': 4.5, 'coordinates': {'latitude': 27.3539547622204, 'longitude': -98.1241938471794}, 'transactions': [], 'price': '$', 'location': {'address1': '725 S Broadway St', 'address2': '', 'address3': '', 'city': 'Premont', 'zip_code': '78375', 'country': 'US', 'state': 'TX', 'display_address': ['725 S Broadway St', 'Premont, TX 78375']}, 'phone': '+13613483747', 'display_phone': '(361) 348-3747', 'distance': 39377.16180697129}, {'id': 'godfathers-pizza-falfurrias', 'name': "Godfather's Pizza", 'image_url': 'https://s3-media1.f

In [194]:
print(len(Restaurant_zip))
print(len(Restaurant_Name))
print(len(Restaurant_City))
print(len(Restaurant_State))
print(len(Restaurant_zip))
print(len(Restaurant_type))
print(len(Restaurant_County))     
print(len(Restaurant_ReviewCount))  
print(len(Restaurant_price_cat))


75
75
75
75
75
75
75
75
75


In [195]:
#print(len(Restaurant_rating))
# len(Restaurant_type)
# len(Restaurant_County)
# len(Restaurant_ReviewCount)

Yelp_df=pd.DataFrame({"Restaurant Name":Restaurant_Name,"Restaurant City":Restaurant_City,"State":Restaurant_State,
                     "ZipCode":Restaurant_zip,"Type":Restaurant_type,
                      "Rating":Restaurant_rating,"County":Restaurant_County,"Review Count":Restaurant_ReviewCount})
Yelp_df

,County,Rating,Restaurant City,Restaurant Name,Review Count,State,Type,ZipCode
0,Conecuh,5.0,Evergreen,The Shack,30,AL,Southern,36401
1,Conecuh,3.5,Evergreen,Shrimp Basket,54,AL,Seafood,36401
2,Conecuh,4.5,Monroeville,Big D's Butts 'N Stuff,7,AL,Barbeque,36460
3,Greene,4.5,Tuscaloosa,Nick's Original Filet House,63,AL,Steakhouses,35401
4,Greene,4.5,Livingston,Taste of India,37,AL,Indian,35470
5,Greene,5.0,York,Larkin's Restaurant & Deli,19,AL,Delis,36925
6,Perry,4.5,Selma,Lannie's Bar-B-Q Spot,34,AL,Barbeque,36703
7,Perry,4.5,Selma,Side Porch Sandwichs and Pub,11,AL,Sandwiches,36703
8,Perry,4.5,Selma,The Coffee Shoppe,11,AL,Coffee & Tea,36701
9,Sumter,4.5,Livingston,Taste of India,37,AL,Indian,35470


In [153]:
data=search(api_key,"Restaurant","Perry, Alabama")


Querying https://api.yelp.com/v3/businesses/search ...


In [154]:
data

{'businesses': [{'categories': [{'alias': 'bbq', 'title': 'Barbeque'}],
   'coordinates': {'latitude': 32.4186935424805,
    'longitude': -87.0103454589844},
   'display_phone': '(334) 874-4478',
   'distance': 31102.075598935935,
   'id': 'lannies-bar-b-q-spot-selma',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/xh_9SZja0VR0BTJrIJcrbQ/o.jpg',
   'is_closed': False,
   'location': {'address1': '2115 Minter Ave',
    'address2': '',
    'address3': '',
    'city': 'Selma',
    'country': 'US',
    'display_address': ['2115 Minter Ave', 'Selma, AL 36703'],
    'state': 'AL',
    'zip_code': '36703'},
   'name': "Lannie's Bar-B-Q Spot",
   'phone': '+13348744478',
   'price': '$',
   'rating': 4.5,
   'review_count': 34,
   'transactions': [],
   'url': 'https://www.yelp.com/biz/lannies-bar-b-q-spot-selma?adjust_creative=-7_tzeiRkxmaXZ-G7LcLVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-7_tzeiRkxmaXZ-G7LcLVw'},
  {'categories': [{'alias': 'sandwiches

In [ ]:
data["businesses"][49]

In [156]:
#pull data from json
print(data["businesses"][2]["name"])
print(data["businesses"][2]["location"]["city"])
print(data["businesses"][2]["location"]["state"])
print(data["businesses"][2]["location"]["zip_code"])
print(data["businesses"][2]["price"])
print(data["businesses"][2]["review_count"])
print(data["businesses"][2]["rating"])
print(data["businesses"][2]["categories"][0]["title"])

The Coffee Shoppe
Selma
AL
36701
$
11
4.5
Coffee & Tea
